In [3]:

import os
import time
from datetime import datetime, timedelta
from collections import defaultdict

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse as dateparse
from tqdm import tqdm


MATCHES_CSV = "historical_matches.csv"   
OUTPUT_CLEANED = "cleaned_worldcup_matches.csv"
OUTPUT_SQUAD_AGGR = "team_squad_aggregates.csv"


TEAM_URLS = {

}


REQUEST_HEADERS = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36"}
REQUEST_DELAY = 3



def safe_parse_date(x):
    """Try to parse a date string to datetime; return NaT on failure."""
    try:
        return dateparse(str(x), dayfirst=False)
    except Exception:
        return pd.NaT

def standardize_team_name(name):
    """Basic team name normalization - extend as required."""
    if pd.isna(name):
        return name
    n = str(name).strip()
    mapping = {
        "USA": "United States",
        "U.S.": "United States",
        "United States of America": "United States",
        "Korea Republic": "South Korea",
        "Korea, Republic of": "South Korea",
        "Iran IR": "Iran",
       
    }
    return mapping.get(n, n)


print("Loading matches CSV:", MATCHES_CSV)
df = pd.read_csv(MATCHES_CSV)
print("Initial rows:", len(df))
print("Columns:", df.columns.tolist())


print("\n--- Basic cleaning ---")


before = len(df)
df = df.drop_duplicates()
print(f"Dropped duplicates: {before - len(df)} rows")


df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]


expected_cols = ["date", "home_team", "away_team", "home_score", "away_score", "competition", "year"]

col_map = {}
if "team1" in df.columns and "team2" in df.columns:
    col_map.update({"team1": "home_team", "team2": "away_team",
                    "score1": "home_score", "score2": "away_score"})

df = df.rename(columns=col_map)


if "date" in df.columns:
    df["date"] = df["date"].apply(safe_parse_date)
else:
    
    if "year" in df.columns:
        df["date"] = pd.to_datetime(df["year"].astype(str) + "-07-01", errors="coerce")
    else:
        df["date"] = pd.NaT


if "home_team" in df.columns and "away_team" in df.columns:
    df["home_team"] = df["home_team"].apply(standardize_team_name)
    df["away_team"] = df["away_team"].apply(standardize_team_name)
else:
    raise ValueError("CSV must contain home_team and away_team columns (or mapped equivalents).")

for col in ["home_score", "away_score"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].fillna(0).astype(int)

if "year" not in df.columns:
    df["year"] = df["date"].dt.year.fillna(df["date"].dt.year.min()).astype(int)

print("After cleaning, rows:", len(df))
print(df.head(3).T)


print("\n--- Feature engineering (match-level) ---")


df["goal_diff_home"] = df["home_score"] - df["away_score"]
df["goal_diff_away"] = -df["goal_diff_home"]


def outcome_label(row):
    if row["home_score"] > row["away_score"]:
        return "home_win"
    elif row["home_score"] < row["away_score"]:
        return "away_win"
    else:
        return "draw"

df["match_outcome"] = df.apply(outcome_label, axis=1)

# 5.3 Simple home advantage binary (if tournament matches may not have 'home' - keep but mark 0 if neutral)
df["home_advantage"] = 1  # default
if "neutral" in df.columns:
    # some datasets have neutral column
    df["home_advantage"] = (~df["neutral"]).astype(int)
else:
    # Heuristic: if competition is "World Cup" and both teams are intercontinental, many matches are neutral -> set 0 if needed
    # For now keep 1; user may adjust based on available fields
    pass

# ----------------------------
# Step 6: Rolling team stats (form, win rate) — requires match history per team
# ----------------------------
print("\n--- Computing team form and win rates ---")

# We'll build a long-form table with each team as a row for each match (so we can compute last-N stats easily)
home = df[["date", "home_team", "away_team", "home_score", "away_score", "match_outcome"]].copy()
home = home.rename(columns={"home_team": "team", "away_team": "opponent",
                            "home_score": "goals_for", "away_score": "goals_against"})
# assign result from team's viewpoint
def result_from_perspective(row):
    if row["goals_for"] > row["goals_against"]:
        return "win"
    elif row["goals_for"] < row["goals_against"]:
        return "loss"
    else:
        return "draw"
home["result"] = home.apply(result_from_perspective, axis=1)

away = df[["date", "away_team", "home_team", "away_score", "home_score", "match_outcome"]].copy()
away = away.rename(columns={"away_team": "team", "home_team": "opponent",
                            "away_score": "goals_for", "home_score": "goals_against"})
away["result"] = away.apply(result_from_perspective, axis=1)

# Long form
long = pd.concat([home, away], ignore_index=True)
long = long.sort_values(["team", "date"]).reset_index(drop=True)

# function to compute last-N form and win rate (last 10 matches)
def compute_last_n_stats(long_df, n=10):
    rows = []
    for team, group in tqdm(long_df.groupby("team"), desc="teams"):
        group = group.sort_values("date").reset_index(drop=True)
        wins_list = []
        for idx, row in group.iterrows():
            # consider last n matches *before* this match
            start = max(0, idx - n)
            prev = group.loc[start:idx-1]
            wins = (prev["result"] == "win").sum()
            draws = (prev["result"] == "draw").sum()
            losses = (prev["result"] == "loss").sum()
            total_prev = len(prev)
            win_rate = wins / total_prev if total_prev > 0 else np.nan
            form_score = (wins * 3 + draws * 1) / (3 * max(1, n))  # normalized
            rows.append({
                "team": team,
                "date": row["date"],
                "wins_last{}_count".format(n): wins,
                "draws_last{}_count".format(n): draws,
                "losses_last{}_count".format(n): losses,
                "win_rate_last{}".format(n): win_rate,
                "form_score_last{}".format(n): form_score
            })
    return pd.DataFrame(rows)

last10_stats = compute_last_n_stats(long, n=10)

# merge last10 stats back to matches for both home and away teams
# merge for home
df = df.merge(last10_stats.rename(columns={
    "date": "date", "team": "home_team"
}), how="left", on=["home_team", "date"])
# rename added cols
home_cols = [c for c in df.columns if c.startswith("wins_last10_count") or c.startswith("win_rate_last10") or c.startswith("form_score_last10")]
# merge for away (we need to merge separately)
df = df.merge(last10_stats.rename(columns={"team": "away_team"}), how="left", on=["away_team", "date"], suffixes=("", "_away"))

# To keep things tidy, produce consistent column names for home and away features
# If merges created duplicated names, handle appropriately (user can tailor final names)

# ----------------------------
# Step 7: Incorporate external features (FIFA ranking, if available)
# ----------------------------
print("\n--- Incorporating FIFA ranking (if available) ---")
# If you obtained a fifa rankings CSV (e.g., fifa_rankings.csv with columns ['team','date','rank']), merge here.
# For this example we'll create a synthetic ranking placeholder if not available.

if False:
    # Example merge if you have a ranking file:
    fifa_df = pd.read_csv("fifa_rankings.csv", parse_dates=["date"])
    # join nearest ranking date before the match date (left join by asof)
    # (implementation would use pd.merge_asof after sorting)
else:
    # Create placeholder ranking: random or based on historical goals - here we'll compute a naive strength metric:
    team_strength = long.groupby("team").agg(total_goals=("goals_for", "sum"),
                                             matches_played=("goals_for", "count")).reset_index()
    team_strength["naive_rank_score"] = team_strength["total_goals"] / team_strength["matches_played"].replace(0, np.nan)
    # Merge back as a "fifa_rank_proxy"
    df = df.merge(team_strength[["team", "naive_rank_score"]].rename(columns={"team": "home_team", "naive_rank_score": "home_rank_proxy"}),
                  how="left", on="home_team")
    df = df.merge(team_strength[["team", "naive_rank_score"]].rename(columns={"team": "away_team", "naive_rank_score": "away_rank_proxy"}),
                  how="left", on="away_team")

# Compute ranking difference feature
df["ranking_diff_proxy"] = df["home_rank_proxy"] - df["away_rank_proxy"]

# ----------------------------
# Step 8: Custom web scraper to fetch squad-level data (Transfermarkt example)
# ----------------------------

def parse_transfermarkt_squad(soup):
    """
    Parse a Transfermarkt squad page soup to extract player rows.
    This function depends on Transfermarkt structure and may need updates.
    Returns list of dicts: [{'player':..., 'age':..., 'caps':..., 'value':...}, ...]
    """
    players = []
    # Transfermarkt uses a table with class 'items'. Rows have classes 'odd'/'even' (subject to change)
    table = soup.find("table", {"class": "items"})
    if table is None:
        return players
    rows = table.find_all("tr", {"class": ["odd", "even"]})
    for r in rows:
        cols = r.find_all("td")
        # Defensive: ensure we have expected columns
        if len(cols) < 5:
            continue
        # Example: name may be in col index 1, age at col 2, caps maybe in col 4, value in last col
        try:
            name = cols[1].get_text(strip=True)
            # age sometimes contains "(age)" formatting - extract digits
            age_text = cols[2].get_text(strip=True)
            age = int(''.join(ch for ch in age_text if ch.isdigit()) or 0)
            # caps may not be present; attempt to parse
            caps_text = cols[3].get_text(strip=True)
            caps = int(''.join(ch for ch in caps_text if ch.isdigit()) or 0)
            value_text = cols[-1].get_text(strip=True)
            # normalize values like "€10.00m" to numeric - simplistic approach:
            if 'm' in value_text.lower():
                val = value_text.lower().replace('€', '').replace('m', '').replace(',', '.').strip()
                market_value = float(val) * 1_000_000
            else:
                market_value = 0.0
        except Exception:
            continue
        players.append({"player": name, "age": age, "caps": caps, "market_value": market_value})
    return players

def scrape_team_transfermarkt(url, team_name, delay=REQUEST_DELAY):
    """
    Scrape the team squad page and return a DataFrame of players.
    WARNING: Make sure you are allowed to scrape the target site; respect robots.txt & site TOS.
    """
    print(f"Scraping {team_name} from {url}")
    time.sleep(delay)
    r = requests.get(url, headers=REQUEST_HEADERS, timeout=20)
    if r.status_code != 200:
        print("Failed to fetch", url, "status:", r.status_code)
        return pd.DataFrame(columns=["team","player","age","caps","market_value"])
    soup = BeautifulSoup(r.text, "html.parser")
    players = parse_transfermarkt_squad(soup)
    if not players:
        print("No player rows found for", team_name)
    dfp = pd.DataFrame(players)
    dfp["team"] = team_name
    return dfp[["team","player","age","caps","market_value"]]

# Example: scrape all TEAM_URLS
if TEAM_URLS:
    all_players = []
    for t, url in TEAM_URLS.items():
        try:
            dfp = scrape_team_transfermarkt(url, t)
            all_players.append(dfp)
        except Exception as e:
            print("Error scraping", t, e)
    squad_df = pd.concat(all_players, ignore_index=True) if all_players else pd.DataFrame(columns=["team","player","age","caps","market_value"])
else:
    print("No TEAM_URLS provided; skipping scraping step.")
    squad_df = pd.DataFrame(columns=["team","player","age","caps","market_value"])

# ----------------------------
# Step 9: Aggregate squad-level metrics to team-level
# ----------------------------
print("\n--- Aggregating squad-level features ---")
if not squad_df.empty:
    squad_df["age"] = pd.to_numeric(squad_df["age"], errors="coerce").fillna(0).astype(int)
    squad_df["caps"] = pd.to_numeric(squad_df["caps"], errors="coerce").fillna(0).astype(int)
    squad_df["market_value"] = pd.to_numeric(squad_df["market_value"], errors="coerce").fillna(0.0)

    team_agg = squad_df.groupby("team").agg(
        avg_age=("age", "mean"),
        median_age=("age", "median"),
        avg_caps=("caps", "mean"),
        pct_experienced=("caps", lambda s: (s >= 50).sum() / max(1, s.count())),
        total_market_value=("market_value", "sum"),
        squad_size=("player", "count")
    ).reset_index()

    # Save aggregate table
    team_agg.to_csv(OUTPUT_SQUAD_AGGR, index=False)
    print("Saved squad aggregates to", OUTPUT_SQUAD_AGGR)
else:
    print("Squad dataframe empty - creating empty team_agg")
    team_agg = pd.DataFrame(columns=["team","avg_age","avg_caps","pct_experienced","total_market_value","squad_size"])

# ----------------------------
# Step 10: Merge team-level aggregates to matches
# ----------------------------
print("\n--- Merging squad aggregates to match table ---")
# Merge home team aggregates
df = df.merge(team_agg.rename(columns={
    "team": "home_team",
    "avg_age": "home_avg_age",
    "avg_caps": "home_avg_caps",
    "pct_experienced": "home_pct_experienced",
    "total_market_value": "home_total_market_value",
    "squad_size": "home_squad_size"
}), how="left", on="home_team")

# Merge away team aggregates
df = df.merge(team_agg.rename(columns={
    "team": "away_team",
    "avg_age": "away_avg_age",
    "avg_caps": "away_avg_caps",
    "pct_experienced": "away_pct_experienced",
    "total_market_value": "away_total_market_value",
    "squad_size": "away_squad_size"
}), how="left", on="away_team")

# Feature: difference in avg age and experience
df["avg_age_diff"] = df["home_avg_age"] - df["away_avg_age"]
df["avg_caps_diff"] = df["home_avg_caps"] - df["away_avg_caps"]
df["market_value_diff"] = df["home_total_market_value"] - df["away_total_market_value"]
df["pct_experienced_diff"] = df["home_pct_experienced"] - df["away_pct_experienced"]

# ----------------------------
# Step 11: Final cleaning and saving
# ----------------------------
print("\n--- Final cleaning and saving ---")
# Drop temporary or highly sparse columns if any
# For example drop columns with all-NaN
df = df.loc[:, df.columns.notna()]
df = df.dropna(axis=1, how="all")

# Optionally encode match_outcome as numeric labels for ML: home_win=1, draw=0, away_win=-1 (example)
df["outcome_numeric"] = df["match_outcome"].map({"home_win": 1, "draw": 0, "away_win": -1})

# Save cleaned dataset
df.to_csv(OUTPUT_CLEANED, index=False)
print("Saved cleaned matches to", OUTPUT_CLEANED)

# Save squad aggregates if produced
if not team_agg.empty:
    team_agg.to_csv(OUTPUT_SQUAD_AGGR, index=False)
    print("Saved team squad aggregates to", OUTPUT_SQUAD_AGGR)

print("\nDone. Summary:")
print("Total matches (final):", len(df))
print("Example columns:", df.columns.tolist()[:30])


Loading matches CSV: historical_matches.csv
Initial rows: 41586
Columns: ['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral']

--- Basic cleaning ---
Dropped duplicates: 0 rows
After cleaning, rows: 41586
                              0                    1                    2
date        1872-11-30 00:00:00  1873-03-08 00:00:00  1874-03-07 00:00:00
home_team              Scotland              England             Scotland
away_team               England             Scotland              England
home_score                    0                    4                    2
away_score                    0                    2                    1
tournament             Friendly             Friendly             Friendly
city                    Glasgow               London              Glasgow
country                Scotland              England             Scotland
neutral                   False                False                False
y

teams: 100%|██████████| 312/312 [01:05<00:00,  4.74it/s]



--- Incorporating FIFA ranking (if available) ---
No TEAM_URLS provided; skipping scraping step.

--- Aggregating squad-level features ---
Squad dataframe empty - creating empty team_agg

--- Merging squad aggregates to match table ---

--- Final cleaning and saving ---
Saved cleaned matches to cleaned_worldcup_matches.csv

Done. Summary:
Total matches (final): 41794
Example columns: ['date', 'home_team', 'away_team', 'home_score', 'away_score', 'tournament', 'city', 'country', 'neutral', 'year', 'goal_diff_home', 'goal_diff_away', 'match_outcome', 'home_advantage', 'wins_last10_count', 'draws_last10_count', 'losses_last10_count', 'win_rate_last10', 'form_score_last10', 'wins_last10_count_away', 'draws_last10_count_away', 'losses_last10_count_away', 'win_rate_last10_away', 'form_score_last10_away', 'home_rank_proxy', 'away_rank_proxy', 'ranking_diff_proxy', 'outcome_numeric']
